# Tutorial to read a Galacticus HDF5 file.

In [1]:
import numpy as np

# Load GalacticusHDF5 class for reading/writing of Galacticus HDF5 files.
from galacticus.io import GalacticusHDF5

In [2]:
# Load Galacticus HDF5 file
ifile = "../galacticus/data/exampleRuns/galacticus.snapshotExample.hdf5"
GH5 = GalacticusHDF5(ifile,'r')

In [3]:
# The Galacticus HDF5 class has an attribute "outputs" that stores the information for the snapshots in this file.
for i in range(len(GH5.outputs.z)):
    print(str(GH5.outputs.iout[i])+" "+GH5.outputs.name[i]+" z = "+str(GH5.outputs.z[i]))

1 Output1 z = 2.0
2 Output2 z = 1.0
3 Output3 z = 0.0


In [4]:
# These outputs are sub-directories in the HDF5 file where the galaxy data is stored. In each output the galaxy 
# data is stored in a sub-directory 'nodeData'.
print GH5.lsGroups("/")
print GH5.lsGroups("/Outputs")
print GH5.lsGroups("/Outputs/Output1")

['Build', 'Filters', 'Outputs', 'Parameters', 'Version', 'globalHistory', 'metaData']
['Output1', 'Output2', 'Output3']
['nodeData']


In [5]:
# For any redshift we can identify the snapshot output that is closest in redshift. For example, for a redshift 
# z = 1.3 we can find the redshift of the nearest snapshot to be:
z = 1.3
zsnap = GH5.nearestRedshift(z)
outputName = GH5.nearestOutputName(z)
print("Snapshot with nearest redshift is "+outputName+" with z = "+str(zsnap))

Snapshot with nearest redshift is Output2 with z = 1.0


In [6]:
# We can then select this snapshot output.
output = GH5.selectOutput(z)
print output.name

/Outputs/Output2


In [7]:
# We can check to see how many galaxies were output in this snapshot.
ngals = GH5.countGalaxies(z)
print("Number of galaxies = "+str(ngals))

Number of galaxies = 160


In [8]:
# Or, if we do not specifiy a redshift, we can count and sum the galaxies in all of the snapshots.
ngals = GH5.countGalaxies()
print("Total number of galaxies = "+str(ngals))

Total number of galaxies = 485


In [9]:
# We can view the galaxy datasets in an output in three ways. Firstly we can do (printing only the first 10 properties):
datasets = output["nodeData"].keys()
print datasets[:10]

[u'basicMass', u'basicTimeLastIsolated', u'blackHoleAccretionRate', u'blackHoleCount', u'blackHoleJetPower', u'blackHoleMass', u'blackHoleRadiativeEfficiency', u'blackHoleSpin', u'darkMatterProfileScale', u'diskAbundancesGasMetals']


In [10]:
# Secondly we can do:
datasets = GH5.lsDatasets(output.name+"/nodeData")
print datasets[:10]

['basicMass', 'basicTimeLastIsolated', 'blackHoleAccretionRate', 'blackHoleCount', 'blackHoleJetPower', 'blackHoleMass', 'blackHoleRadiativeEfficiency', 'blackHoleSpin', 'darkMatterProfileScale', 'diskAbundancesGasMetals']


In [11]:
# Or thirdly we can do:
datasets = GH5.availableDatasets(z)
print datasets[:10]

['basicMass', 'basicTimeLastIsolated', 'blackHoleAccretionRate', 'blackHoleCount', 'blackHoleJetPower', 'blackHoleMass', 'blackHoleRadiativeEfficiency', 'blackHoleSpin', 'darkMatterProfileScale', 'diskAbundancesGasMetals']


In [12]:
# Reading galaxy proprerties can be done by directly reading from the HDF5 output, for example:
arr = np.array(output["nodeData/basicMass"])
print arr[:10]

[  4.57338452e+10   3.15216623e+10   6.13070521e+10   4.95172370e+10
   3.30114890e+10   3.30239637e+11   4.09871892e+10   9.48560580e+10
   3.30114890e+10   3.58090668e+10]


In [13]:
# Or by using the 'readGalaxies' function, where the 'props' keyword is a list of the properties that one
# wishes to read.
arr = GH5.readGalaxies(z,props=['basicMass'])
print type(arr)
print arr["basicMass"][:10]

<type 'numpy.ndarray'>
[  4.57338452e+10   3.15216623e+10   6.13070521e+10   4.95172370e+10
   3.30114890e+10   3.30239637e+11   4.09871892e+10   9.48560580e+10
   3.30114890e+10   3.58090668e+10]


In [14]:
# Using the 'readGalaxies' function, multiple properties can be read at once. Wildcards can be used to select
# multiple properties with similar names. The data is extracted into a numpy structured array.
arr = GH5.readGalaxies(z,props=['basicMass','blackHoleCount','*MassStellar'])
print type(arr)
print arr.dtype.names
print arr['diskMassStellar'][:10]

<type 'numpy.ndarray'>
('basicMass', 'blackHoleCount', 'diskMassStellar', 'spheroidMassStellar')
[  6.48920984e+08   5.65349962e+07   9.37697597e+08   3.30224188e+08
   1.89293148e+08   8.21404327e+09   2.80909787e+08   7.98192152e+08
   3.02719122e+08   2.13286167e+08]


In [15]:
# The Galacticus file is then closed using:
GH5.close()